# IceNet Library Usage

## Context

### Purpose
This notebook demonstrates the use of the [IceNet library](https://pypi.org/project/icenet/) for sea-ice forecasting trained using climate reanalysis and observational data.

### Description
[IceNet](https://github.com/icenet-ai/icenet/) is a python library that provides the ability to download, process, train and predict from end to end. Users can interact with IceNet either via the python interface or via a set of command-line interfaces (CLI) which provide a high-level interface that covers the above abilities.

This notebook demonstrates the use of the python library api for forecasting sea ice for a reduced dataset to demonstrate its capabilities. The final output of interest are maps of sea ice concentration.

### Modelling approach
This modelling approach allows users to immediately utilise the library for producing sea ice concentration forecasts.

### Highlights
The key features of an end to end run are: 
* Setup: _this was concerned with setting up the conda environment, which remains the same as in 01.cli_demonstration_
* [1. Introduction](#1-Introduction) the environment and project structure.
* [2. Download](#2-Download) sea ice concentration data as training data.
* [3. Process](#3-Process) downloaded data, and generate cached datasets to speed up training.
* [4. Train](#4-Train) the neural network and generate checkpoint and model output.
* [5. Predict](#5-Predict) for defined dates.
* [6. Visualisation](#6-Visualisation) of the prediction output.

_This follows the same structure as the CLI demonstration notebook so that it's easy to follow step-by-step..._

### Contributions
#### Notebook
James Byrne (author)

Bryn Noel Ubald

__Please raise issues [in this repository](https://github.com/icenet-ai/icenet-notebooks/issues) to suggest updates to this notebook!__ 

Contact me at _jambyr \<at\> bas.ac.uk_ for anything else...

#### Modelling codebase
James Byrne (code author), Tom Andersson (science author)

#### Modelling publications
Andersson, T.R., Hosking, J.S., Pérez-Ortiz, M. et al. Seasonal Arctic sea ice forecasting with probabilistic deep learning. Nat Commun 12, 5124 (2021). https://doi.org/10.1038/s41467-021-25257-4

#### Involved organisations
The Alan Turing Institute and British Antarctic Survey

___
## 1. Introduction

Once installed the API can be utilised as easily as the CLI commands from a shell, via any Python interpreter. As usual ensure that you're operating within the conda environment you installed the library into.

### A tip on CLI - API usage

All of the `icenet_*` CLI commands behind the scenes implement API activities. By inspecting the [`setup.py` ](https://github.com/JimCircadian/icenet2/blob/main/setup.py#L32) entry points you can locate the module and thus the code used by these. 

In most cases the CLI imposes various assumptions about what to do without exposing, necessarily, all available options to change the behaviour of the library. This is primarily as the CLI entry points are still under development to open up the options, so these CLI operations are for introductory use and API usage is recommended for advanced use cases and pipeline integrations.

### What we'll cover

For the sake of illustration this notebook will display and execute the equivalent API code, equivalent to [the first notebook of this collection](01.cli_demonstration.ipynb) as well as some updates that incorporate the visualisations from [the third notebook describing the data](03.data_and_forecasts.ipynb). However, for the sake of extending our dataset, we'll work towards extending our original downloads from covering *2019-12-28 through 2020-04-30* to cover 2020 in totality, as well as creating a more complex selection of dates for our dataset, training and predicting with new networks.

In [2]:
import numpy as np
import pandas as pd
import os
import random

# We also set the logging level so that we get some feedback from the API
import logging
logging.basicConfig(level=logging.INFO)

___
## 2. Download

The following is preparation of the downloaders, whose instantiation describes the interactions with the upstream APIs/data interfaces used to source various types of data. 

In this section, we download all required data with our extended date range. All downloaders inherit a `download` method from the `Downloader` class in [`icenet.data.producers`](https://github.com/icenet-ai/icenet/blob/main/icenet/data/producers.py), which also contains two other data producing classes `Generator` (which Masks inherits from) and `Processor` (used in the next section), each providing abstract implementations that multiple classes derive from.

In [3]:
from icenet.data.sic.mask import Masks
from icenet.data.interfaces.cds import ERA5Downloader
from icenet.data.sic.osisaf import SICDownloader

Next we download all required data with our extended date range. All downloaders inherit a `download` method from the `Downloader` class in [`icenet.data.producers`](https://github.com/icenet-ai/icenet/blob/main/icenet/data/producers.py), which also contains two other data producing classes `Generator` (which Masks inherits from) and `Processor` (used in the next section), each providing abstract implementations that multiple classes derive from.

### Mask data

We start here with generating the masks for training/prediction. This includes regions where sea ice does not form, land regions, and the [polar hole](https://blogs.egu.eu/divisions/cr/2016/10/14/image-of-the-week-the-polar-hole/).

We saw in [01.cli_demonstration](01.cli_demonstration.ipynb), that we can generate this using the `icenet_data_masks` CLI, but to do this in Python, we can do the following:

In [4]:
masks = Masks(north=False, south=True)
masks.generate(save_polarhole_masks=False)

INFO:root:siconca ice_conc_sh_ease2-250_cdr-v2p0_200001021200.nc already exists
INFO:root:Saving ./data/masks/south/masks/active_grid_cell_mask_01.npy
INFO:root:siconca ice_conc_sh_ease2-250_cdr-v2p0_200002021200.nc already exists
INFO:root:Saving ./data/masks/south/masks/active_grid_cell_mask_02.npy
INFO:root:siconca ice_conc_sh_ease2-250_cdr-v2p0_200003021200.nc already exists
INFO:root:Saving ./data/masks/south/masks/active_grid_cell_mask_03.npy
INFO:root:siconca ice_conc_sh_ease2-250_cdr-v2p0_200004021200.nc already exists
INFO:root:Saving ./data/masks/south/masks/active_grid_cell_mask_04.npy
INFO:root:siconca ice_conc_sh_ease2-250_cdr-v2p0_200005021200.nc already exists
INFO:root:Saving ./data/masks/south/masks/active_grid_cell_mask_05.npy
INFO:root:siconca ice_conc_sh_ease2-250_cdr-v2p0_200006021200.nc already exists
INFO:root:Saving ./data/masks/south/masks/active_grid_cell_mask_06.npy
INFO:root:siconca ice_conc_sh_ease2-250_cdr-v2p0_200007021200.nc already exists
INFO:root:Savi

### Climate and Sea Ice Data

We saw in [01.cli_demonstration.ipynb](01.cli_demonstration.ipynb) that obtaining and preparing data can be achieved using `icenet_data_*` commands. To do so, you first need to __configure the [CDS API](https://cds.climate.copernicus.eu/) token yourself__ - see [here](https://cds.climate.copernicus.eu/api-how-to) for more instructions on how to set this up.

Here we will download [ERA5 reanalysis](https://cds.climate.copernicus.eu/cdsapp#!/search?type=dataset&keywords=((%20%22Product%20type:%20Reanalysis%22%20))) data using the direct API using the `icenet.data.interfaces.cds.ERA5Downloader` class and [OSISAF sea-ice concentration (SIC) data](https://osisaf-hl.met.no/v2p1-sea-ice-index) using the `icenet.data.sic.osisaf.SICDownloader` class.

In [33]:
era5 = ERA5Downloader(
    var_names=["tas", "zg", "uas", "vas"],      # Name of variables to download
    dates=[                                     # Dates to download the variable data for
        pd.to_datetime(date).date()
        for date in pd.date_range("2020-01-01", "2020-04-30", freq="D")
    ],
    delete_tempfiles=False,                     # Whether to delete temporary downloaded files
    levels=[None, [250, 500], None, None],      # The levels at which to obtain the variables for (e.g. for zg, it is the pressure levels)
    max_threads=64,                             # Maximum number of concurrent downloads
    north=False,                                # Boolean: Whether require data across northern hemisphere
    south=True,                                 # Boolean: Whether require data across southern hemisphere
    # NOTE: there appears to be a bug with the toolbox API at present (icenet#54)
    use_toolbox=False)                          # Experimental, alternative download method

era5.download()                                 # Start downloading

INFO:root:Upping connection limit for max_threads > 10
INFO:root:Building request(s), downloading and daily averaging from ERA5 API
INFO:root:Processing single download for tas @ None with 121 dates
INFO:root:Processing single download for zg @ 250 with 121 dates
INFO:root:Processing single download for zg @ 500 with 121 dates
INFO:root:Processing single download for uas @ None with 121 dates
INFO:root:Processing single download for vas @ None with 121 dates


INFO:root:No requested dates remain, likely already present
INFO:root:No requested dates remain, likely already present
INFO:root:No requested dates remain, likely already present
INFO:root:No requested dates remain, likely already present
INFO:root:No requested dates remain, likely already present
INFO:root:0 daily files downloaded


In [35]:
sic = SICDownloader(
    dates=[
        pd.to_datetime(date).date()     # Dates to download the variable data for
        for date in pd.date_range("2020-01-01", "2020-04-30", freq="D")
    ],
    delete_tempfiles=False,             # Whether to delete temporary downloaded files
    north=False,                        # Boolean: Whether to use mask for this region
    south=True,                         # Boolean: Whether to use mask for this region
    parallel_opens=False,               # Boolean: Whether to use `dask.delayed` to open and preprocess multiple files in parallel
)

sic.download()

INFO:root:Downloading SIC datafiles to .temp intermediates...
INFO:root:Excluding 121 dates already existing from 121 dates requested.
INFO:root:Opening for interpolation: ['./data/osisaf/south/siconca/2020.nc']


INFO:root:Processing 0 missing dates


The `ERA5Downloader` inherits from `ClimateDownloader`, from which several implementations derive their functionality. Two particularly useful methods shown below allow the downloaded data to be converted to the same grid and orientation as the OSISAF SIC data.

In [36]:
era5.regrid()
era5.rotate_wind_data()

INFO:root:No regrid batches to processing, moving on...
INFO:root:Rotating wind data prior to merging
INFO:root:Rotating wind data in ./data/era5/south/uas ./data/era5/south/vas
INFO:root:0 files for uas
INFO:root:0 files for vas
INFO:root:Rotating wind data in ./data/era5/south/uas ./data/era5/south/vas
INFO:root:0 files for uas
INFO:root:0 files for vas


It is hopefully obvious now that the CLI operations wrap several activities within the API up for convenience and initial ease of use, but that for experimentation, research and advancing the pipeline the API offers greater flexibility to manipulate processing chains as required for these purposes.

___
## 3. Process

Similarly to the downloaders, each data producer (be it a `Downloader` or `Generator`) has a respective `Processor` that converts the `/data/` products into a normalised, preprocessed dataset under `/processed/` as per the `icenet_process_*` commands.

Firstly, to make life a bit easier, we set up some variables that are normally handled from the CLI arguments. In this case we're splitting the validation and test sets out of the 2020 data in a fairly naive manner.

In [8]:
processing_dates = dict(
    train=[pd.to_datetime(el) for el in pd.date_range("2020-01-01", "2020-03-31")],
    val=[pd.to_datetime(el) for el in pd.date_range("2020-04-03", "2020-04-23")],
    test=[pd.to_datetime(el) for el in pd.date_range("2020-04-01", "2020-04-02")],
)
processed_name = "tutorial_api_data"

Next, we create the data producer and configure them for the dataset we want to create.

In [9]:
from icenet.data.processors.era5 import IceNetERA5PreProcessor
from icenet.data.processors.meta import IceNetMetaPreProcessor
from icenet.data.processors.osi import IceNetOSIPreProcessor

pp = IceNetERA5PreProcessor(
    ["uas", "vas"],                 # Absolute normalised variables
    ["tas", "zg500", "zg250"],      # Variables defined as deviations from an aggregated norm
    processed_name,
    processing_dates["train"],
    processing_dates["val"],
    processing_dates["test"],
    linear_trends=tuple(),
    north=False,
    south=True
)

osi = IceNetOSIPreProcessor(
    ["siconca"],                    # Absolute normalised variables
    [],                             # Variables defined as deviations from an aggregated norm
    processed_name,
    processing_dates["train"],
    processing_dates["val"],
    processing_dates["test"],
    linear_trends=tuple(),
    north=False,
    south=True
)

meta = IceNetMetaPreProcessor(
    processed_name,
    north=False,
    south=True
)

INFO:root:Creating path: ./processed/tutorial_api_data/era5
INFO:root:Creating path: ./processed/tutorial_api_data/osisaf
INFO:root:Creating path: ./processed/tutorial_api_data/meta


Next, we initialise the data processors using `init_source_data` which scans the data source directories to understand what data is available for processing based on the parameters. Since we named the processed data `"tutorial_api_data"` above, it will create a data loader config file, `loader.tutorial_api_data.json`, in the current directory.

In [10]:
pp.init_source_data(
    lag_days=1,
)
pp.process()

osi.init_source_data(
    lag_days=1,
)
osi.process()

meta.process()

INFO:root:Processing 91 dates for train category
INFO:root:Including lag of 1 days
INFO:root:Including lead of 93 days
INFO:root:No data found for 2019-12-31, outside data boundary perhaps?
INFO:root:Processing 21 dates for val category
INFO:root:Including lag of 1 days
INFO:root:Including lead of 93 days
INFO:root:Processing 2 dates for test category
INFO:root:Including lag of 1 days
INFO:root:Including lead of 93 days
INFO:root:Got 1 files for tas
INFO:root:Got 1 files for uas
INFO:root:Got 1 files for vas
INFO:root:Got 1 files for zg250
INFO:root:Got 1 files for zg500
INFO:root:Opening files for uas
INFO:root:Filtered to 121 units long based on configuration requirements
INFO:root:Normalising uas
INFO:root:Opening files for vas
INFO:root:Filtered to 121 units long based on configuration requirements
INFO:root:Normalising vas
INFO:root:Opening files for tas
INFO:root:Filtered to 121 units long based on configuration requirements
INFO:root:Generating climatology ./processed/tutorial_a

At this point the preprocessed data is ready to convert or create a configuration for the network dataset.

### Dataset creation

As with the `icenet_dataset_create` command we can create a dataset configuration for training the network. As before this can include cached data for the network in the format of a TFRecordDataset compatible set of tfrecords. To achieve this we create the `IceNetDataLoader`, which can both generate `IceNetDataSet` configurations (which easily provide the necessary functionality for training and prediction) as well as individual data samples for direct usage.

In [11]:
from icenet.data.loaders import IceNetDataLoaderFactory

implementation = "dask"
loader_config = "loader.tutorial_api_data.json"
dataset_name = "api_dataset"
lag = 1

dl = IceNetDataLoaderFactory().create_data_loader(
    implementation,
    loader_config,
    dataset_name,
    lag,
    n_forecast_days=7,
    north=False,
    south=True,
    output_batch_size=4,
    generate_workers=8)

INFO:root:Creating path: ./network_datasets/api_dataset
INFO:root:Loading configuration loader.tutorial_api_data.json


In [12]:
dl

We can see the loader config contains information about the data sources included and also the different dates to use for the training, validation and test sets:

In [13]:
dl._config

{'sources': {'era5': {'name': 'tutorial_api_data',
   'implementation': 'IceNetERA5PreProcessor',
   'anom': ['tas', 'zg500', 'zg250'],
   'abs': ['uas', 'vas'],
   'dates': {'train': ['2020_01_01',
     '2020_01_02',
     '2020_01_03',
     '2020_01_04',
     '2020_01_05',
     '2020_01_06',
     '2020_01_07',
     '2020_01_08',
     '2020_01_09',
     '2020_01_10',
     '2020_01_11',
     '2020_01_12',
     '2020_01_13',
     '2020_01_14',
     '2020_01_15',
     '2020_01_16',
     '2020_01_17',
     '2020_01_18',
     '2020_01_19',
     '2020_01_20',
     '2020_01_21',
     '2020_01_22',
     '2020_01_23',
     '2020_01_24',
     '2020_01_25',
     '2020_01_26',
     '2020_01_27',
     '2020_01_28',
     '2020_01_29',
     '2020_01_30',
     '2020_01_31',
     '2020_02_01',
     '2020_02_02',
     '2020_02_03',
     '2020_02_04',
     '2020_02_05',
     '2020_02_06',
     '2020_02_07',
     '2020_02_08',
     '2020_02_09',
     '2020_02_10',
     '2020_02_11',
     '2020_02_12',
   

In [14]:
dl._config.keys()

dict_keys(['sources', 'dtype', 'shape', 'missing_dates'])

At this point we can either use `generate` or `write_dataset_config_only` to produce a ready-to-go `IceNetDataSet` configuration. Both of these will generate a dataset config, `dataset_config.api_dataset.json` (recall we set the dataset name as `api_dataset` above).

In [15]:
dl.generate()

INFO:distributed.http.proxy:To route to workers diagnostics web server please install jupyter-server-proxy: python -m pip install jupyter-server-proxy
INFO:distributed.scheduler:State start
INFO:distributed.scheduler:  Scheduler at:     tcp://127.0.0.1:35478
INFO:distributed.scheduler:  dashboard at:  http://127.0.0.1:8888/status
INFO:distributed.scheduler:Registering Worker plugin shuffle
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:40934'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:46140'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:44091'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:44118'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:37370'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:33992'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:34449'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:42675'
INFO:distributed.scheduler:Register wor

INFO:distributed.nanny:Closing Nanny gracefully at 'tcp://127.0.0.1:42675'. Reason: worker-close
INFO:distributed.nanny:Closing Nanny gracefully at 'tcp://127.0.0.1:46140'. Reason: worker-close
INFO:distributed.nanny:Closing Nanny gracefully at 'tcp://127.0.0.1:33992'. Reason: worker-close
INFO:distributed.nanny:Closing Nanny gracefully at 'tcp://127.0.0.1:37370'. Reason: worker-close
INFO:distributed.nanny:Closing Nanny gracefully at 'tcp://127.0.0.1:44118'. Reason: worker-close
INFO:distributed.nanny:Closing Nanny gracefully at 'tcp://127.0.0.1:40934'. Reason: worker-close
INFO:distributed.nanny:Closing Nanny gracefully at 'tcp://127.0.0.1:34449'. Reason: worker-close
INFO:distributed.nanny:Closing Nanny gracefully at 'tcp://127.0.0.1:44091'. Reason: worker-close
INFO:distributed.scheduler:Remove client Client-worker-36d6009e-e7bb-11ee-8c79-c4cbe1af5a66
INFO:distributed.core:Received 'close-stream' from tcp://127.0.0.1:59088; closing.
INFO:distributed.scheduler:Remove client Client-w

To generate samples from this, we can use the `.generate_sample()` method, which returns the inputs `x`, `y` and sample weights `sw`:

In [16]:
x, y, sw = dl.generate_sample(pd.Timestamp("2020-04-01"))

In [17]:
print(f"type(x): {type(x)}, x.shape: {x.shape}")
print(f"type(y): {type(y)}, y.shape: {y.shape}")
print(f"type(sw): {type(sw)}, sw.shape: {sw.shape}")

type(x): <class 'numpy.ndarray'>, x.shape: (432, 432, 9)
type(y): <class 'numpy.ndarray'>, y.shape: (432, 432, 7, 1)
type(sw): <class 'numpy.ndarray'>, sw.shape: (432, 432, 7, 1)


___
## 4. Train

For single runs we programmatically can call the same method used by the CLI. `train_model` defines the training process from start to finish. The [`model-ensembler`](https://github.com/JimCircadian/model-ensembler) works outside the API, controlling multiple CLI submissions. Customising an ensemble can be achieved through looking at the configuration in [the pipeline repository](https://github.com/antarctica/IceNet-Pipeline). That said, if workflow system integration (e.g. Airflow) is desired, integrating via this method is the way to go.

In [18]:

from icenet.data.dataset import IceNetDataSet
import tensorflow as tf

dataset_config = "dataset_config.api_dataset.json"
dataset = IceNetDataSet(dataset_config, batch_size=4)
strategy = tf.distribute.get_strategy()

INFO:root:Loading configuration dataset_config.api_dataset.json
INFO:root:Training dataset path: ./network_datasets/api_dataset/south/train
INFO:root:Validation dataset path: ./network_datasets/api_dataset/south/val
INFO:root:Test dataset path: ./network_datasets/api_dataset/south/test


We can view the loaded dataset configuration.

In [19]:
dataset._config

{'identifier': 'api_dataset',
 'implementation': 'DaskMultiWorkerLoader',
 'channels': ['uas_abs_1',
  'vas_abs_1',
  'siconca_abs_1',
  'tas_anom_1',
  'zg250_anom_1',
  'zg500_anom_1',
  'cos_1',
  'land_1',
  'sin_1'],
 'counts': {'train': 91, 'val': 21, 'test': 2},
 'dtype': 'float32',
 'loader_config': '/data/hpcdata/users/bryald/git/icenet/icenet-notebooks/loader.tutorial_api_data.json',
 'missing_dates': [],
 'n_forecast_days': 7,
 'north': False,
 'num_channels': 9,
 'shape': [432, 432],
 'south': True,
 'dataset_path': './network_datasets/api_dataset',
 'generate_workers': 8,
 'loss_weight_days': True,
 'output_batch_size': 4,
 'var_lag': 1,
 'var_lag_override': {}}

You can obtain the data loader that was used to create the dataset config via the `.get_data_loader()` method:

In [20]:
dataset.loader_config

'/data/hpcdata/users/bryald/git/icenet/icenet-notebooks/loader.tutorial_api_data.json'

You can obtain the data loader that was used to create the dataset config via the `.get_data_loader()` method:

In [21]:
dataset.get_data_loader()

INFO:root:Loading configuration /data/hpcdata/users/bryald/git/icenet/icenet-notebooks/loader.tutorial_api_data.json


We can use `train_model` to train. Note that we can add more arguments just those that can be set with the `icenet_train` CLI, for example: 
- `learning_rate`
- `lr_10e_decay_fac`
- `lr_decay_start`
- `lr_decay_end`

and several more.

In [22]:
from icenet.model.train import train_model

trained_path, history = train_model(
    run_name="api_test_run",
    dataset=dataset,
    epochs=10,
    n_filters_factor=0.6,
    seed=42,
    strategy=strategy,
    training_verbosity=2,
)

INFO:root:Creating network folder: ./results/networks/api_test_run
INFO:root:Adding tensorboard callback


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 432, 432, 9)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 432, 432, 38)         3116      ['input_1[0][0]']             
                                                                                                  
 conv2d_1 (Conv2D)           (None, 432, 432, 38)         13034     ['conv2d[0][0]']              
                                                                                                  
 batch_normalization (Batch  (None, 432, 432, 38)         152       ['conv2d_1[0][0]']            
 Normalization)                                                                               

INFO:root:Datasets: 23 train, 6 val and 1 test filenames
INFO:root:Reducing datasets to 1.0 of total files
INFO:root:Reduced: 23 train, 6 val and 1 test filenames
INFO:root:
Setting learning rate to: 9.999999747378752e-05



Epoch 1/10


I0000 00:00:1711050255.495952   37966 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


...

INFO:root:
Setting learning rate to: 9.999999747378752e-05



Epoch 9/10

Epoch 9: val_rmse improved from 35.19866 to 33.67870, saving model to ./results/networks/api_test_run/api_test_run.network_api_dataset.42.h5
23/23 - 15s - loss: 12.0957 - binacc: 96.5316 - mae: 3.7294 - rmse: 8.1568 - mse: 1527.5958 - val_loss: 206.2061 - val_binacc: 38.7499 - val_mae: 31.7392 - val_rmse: 33.6787 - val_mse: 1781.4158 - lr: 1.0000e-04 - 15s/epoch - 652ms/step


INFO:root:
Setting learning rate to: 9.999999747378752e-05



Epoch 10/10

Epoch 10: val_rmse improved from 33.67870 to 27.25531, saving model to ./results/networks/api_test_run/api_test_run.network_api_dataset.42.h5
23/23 - 15s - loss: 12.0743 - binacc: 96.4211 - mae: 3.7555 - rmse: 8.1496 - mse: 1559.6680 - val_loss: 135.0495 - val_binacc: 45.1558 - val_mae: 25.2393 - val_rmse: 27.2553 - val_mse: 1709.8202 - lr: 1.0000e-04 - 15s/epoch - 645ms/step


INFO:root:Saving network to: ./results/networks/api_test_run/api_test_run.network_api_dataset.42.h5
INFO:tensorflow:Assets written to: ./results/networks/api_test_run/api_test_run.model_api_dataset.42/assets


Breaking `train_model` apart, one can look at customising the training process itself programmatically. Here, we've reduced `train_model` to its component parts with some notes about missing items (e.g. callbacks and wandb integration), to give some insight into how the training workflow is architected.

In [23]:
from icenet.data.dataset import IceNetDataSet
from icenet.model.models import unet_batchnorm
import icenet.model.losses as losses
import icenet.model.metrics as metrics

# train_model sets up wandb and attempts seeding here (see icenet#8 for issues around multi-GPU determinism)
seed = 45
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
random.seed(seed)
tf.random.set_seed(seed)
tf.keras.utils.set_random_seed(seed)

# initialise IceNetDataSet
ds = IceNetDataSet(dataset_config, batch_size=4)

input_shape = (*ds.shape, ds.num_channels)
train_ds, val_ds, test_ds = ds.get_split_datasets()

# train_model handles pickup runs/trained networks
run_name = "custom_run"
network_folder = os.path.join(".", "results", "networks", run_name)

if not os.path.exists(network_folder):
    logging.info("Creating network folder: {}".format(network_folder))
    os.makedirs(network_folder)

network_path = os.path.join(network_folder,
                            "{}.network_{}.{}.h5".format(run_name,
                                                         ds.identifier,
                                                         seed))

callbacks_list = list()
# train_model sets up various callbacks: early stopping, lr scheduler, 
# checkpointing, wandb and tensorboard

with strategy.scope():
    loss = losses.WeightedMSE()
    metrics_list = [
        metrics.WeightedMAE(),
        metrics.WeightedRMSE(),
        losses.WeightedMSE()
    ]

    network = unet_batchnorm(
        input_shape=input_shape,
        loss=loss,
        metrics=metrics_list,
        learning_rate=1e-4,
        filter_size=3,
        n_filters_factor=0.6,
        n_forecast_days=ds.n_forecast_days,
    )

# train_model loads weights
network.summary()

model_history = network.fit(
    train_ds,
    epochs=5,
    verbose=2,
    callbacks=callbacks_list,
    validation_data=val_ds,
    max_queue_size=10,
)

logging.info("Saving network to: {}".format(network_path))
network.save_weights(network_path)


INFO:root:Loading configuration dataset_config.api_dataset.json
INFO:root:Training dataset path: ./network_datasets/api_dataset/south/train
INFO:root:Validation dataset path: ./network_datasets/api_dataset/south/val
INFO:root:Test dataset path: ./network_datasets/api_dataset/south/test
INFO:root:Datasets: 46 train, 12 val and 2 test filenames
INFO:root:Creating network folder: ./results/networks/custom_run


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 432, 432, 9)]        0         []                            
                                                                                                  
 conv2d_24 (Conv2D)          (None, 432, 432, 38)         3116      ['input_2[0][0]']             
                                                                                                  
 conv2d_25 (Conv2D)          (None, 432, 432, 38)         13034     ['conv2d_24[0][0]']           
                                                                                                  
 batch_normalization_8 (Bat  (None, 432, 432, 38)         152       ['conv2d_25[0][0]']           
 chNormalization)                                                                           

INFO:root:Saving network to: ./results/networks/custom_run/custom_run.network_api_dataset.45.h5


As can be seen the training workflow is very standard for deep learning networks, with `train_model` and CLI wrapping up the training process with a lot of customisation of extraneous functionality. 

___
## 5. Predict

In much the same manner as with `train_model`, the `predict_forecast` method acts as a convenient entry point workflow system integration, CLI entry as well as an overridable method upon which to base custom implementations. Using the method directly relies on loading from a prepared (but perhaps not cached) dataset.

Some parameters are fed to `predict_forecast` that ideally shouldn't need to be specified (like `seed` and `n_filters_factor`) and might seem contextually odd. They're used to locate the appropriate saved network. *This will be cleaned up in a future version*.  

In [24]:
from icenet.model.predict import predict_forecast

# Follows the naming convention used by the CLI version
output_dir = os.path.join(".", "results", "predict",
                          "custom_run_forecast",
                          "{}.{}".format(run_name, "42"))

predict_forecast(
    dataset_config=dataset_config,
    network_name=run_name,
    n_filters_factor=0.6,
    output_folder=output_dir,
    seed=seed,
    start_dates=[pd.to_datetime(el).date()
                 for el in pd.date_range("2020-04-01", "2020-04-02")],
    test_set=True,
)

INFO:root:Loading configuration dataset_config.api_dataset.json
INFO:root:Training dataset path: ./network_datasets/api_dataset/south/train
INFO:root:Validation dataset path: ./network_datasets/api_dataset/south/val
INFO:root:Test dataset path: ./network_datasets/api_dataset/south/test
INFO:root:Loading configuration /data/hpcdata/users/bryald/git/icenet/icenet-notebooks/loader.tutorial_api_data.json
INFO:root:Loading model from ./results/networks/custom_run/custom_run.network_api_dataset.45.h5...
INFO:root:Datasets: 69 train, 18 val and 3 test filenames
INFO:root:Processing test batch 1, item 0 (date 2020-04-01)
INFO:root:Running prediction 2020-04-01
INFO:root:Saving 2020-04-01 - forecast output (1, 432, 432, 7)
INFO:root:Processing test batch 1, item 1 (date 2020-04-02)
INFO:root:Running prediction 2020-04-02
INFO:root:Saving 2020-04-02 - forecast output (1, 432, 432, 7)


The persistence and respective use of these results is then up to the user, with the threefold outputs correlating to that which is normally saved to disk as individual files containing the numpy arrays by the CLI command.

The [internals of the `predict_forecast` method](https://github.com/JimCircadian/icenet2/blob/main/icenet2/model/predict.py#L17) are still undergoing some development, but it should be noted that this method can be easily overridden or called as part of a larger workflow. In particular, within this method it's worth noting the importance of the `testset` parameter. 

Should `testset` be true, then cached data generated in `network_datasets` is never used, and instead the preprocessed data in `processed` is used directly. This actually makes the implementation of `predict_forecast` extremely simple compared with the alternative, due to some outstanding work to derive dates from the cached batched files. 

As before this is revised implementation in order to illustrate the "non-testset" use case, so several modifications are in situ for notebook execution:

In [25]:
from icenet.data.dataset import IceNetDataSet
from icenet.model.models import unet_batchnorm
import tensorflow as tf

start_dates = [el.date() for el in pd.date_range("2020-04-01", "2020-04-02")]

# initialise IceNetDataSet and obtain data loader used to generate the dataset config
ds = IceNetDataSet(dataset_config, batch_size=4)
dl = ds.get_data_loader()

logging.info("Generating forecast inputs from processed/ files")

# generate samples for prediction
forecast_inputs, gen_outputs, sample_weights = \
    list(zip(*[dl.generate_sample(date, prediction=True) for date in start_dates]))

network_folder = os.path.join(".", "results", "networks", "custom_run")

dataset_name = ds.identifier
network_path = os.path.join(network_folder,
                            "{}.network_{}.{}.h5".format(run_name,
                                                         "api_dataset",
                                                         seed))

logging.info("Loading model from {}...".format(network_path))

network = unet_batchnorm(
    (*ds.shape, dl.num_channels),
    [],
    [],
    n_filters_factor=0.6,
    n_forecast_days=ds.n_forecast_days
)
network.load_weights(network_path)

predictions = []

for i, net_input in enumerate(forecast_inputs):
    logging.info("Running prediction {} - {}".format(i, start_dates[i]))
    pred = network(tf.convert_to_tensor([net_input]), training=False)
    predictions.append(pred)

INFO:root:Loading configuration dataset_config.api_dataset.json
INFO:root:Training dataset path: ./network_datasets/api_dataset/south/train
INFO:root:Validation dataset path: ./network_datasets/api_dataset/south/val
INFO:root:Test dataset path: ./network_datasets/api_dataset/south/test
INFO:root:Loading configuration /data/hpcdata/users/bryald/git/icenet/icenet-notebooks/loader.tutorial_api_data.json
INFO:root:Generating forecast inputs from processed/ files
INFO:root:Loading model from ./results/networks/custom_run/custom_run.network_api_dataset.45.h5...
INFO:root:Running prediction 0 - 2020-04-01
INFO:root:Running prediction 1 - 2020-04-02


In [26]:
print(f"Predictions: {len(predictions)}, shape {predictions[0].shape}")
print(f"Generated outputs: {len(gen_outputs)}, shape {gen_outputs[0].shape}")
print(f"Sample weights: {len(sample_weights)}, shape {sample_weights[0].shape}")

Predictions: 2, shape (1, 432, 432, 7)
Generated outputs: 2, shape (432, 432, 7, 1)
Sample weights: 2, shape (432, 432, 7, 1)


To generate __a CF-compliant NetCDF containing the forecasts requested__ we need to run `icenet_output`, these can then be post-processed. As an input to this command, we need to provide it with a `csv` containing the test dates. In this example, we use `printf` to generate the required file.

In [28]:
!printf "2020-04-01\n2020-04-02" | tee predict_dates.csv

2020-04-01
2020-04-02

In [29]:
!icenet_output -m -o ./results/predict custom_run_forecast api_dataset predict_dates.csv

[21-03-24 19:57:43 :INFO    ] - Loading configuration ./dataset_config.api_dataset.json
[21-03-24 19:57:43 :INFO    ] - Training dataset path: ./network_datasets/api_dataset/south/train
[21-03-24 19:57:43 :INFO    ] - Validation dataset path: ./network_datasets/api_dataset/south/val
[21-03-24 19:57:43 :INFO    ] - Test dataset path: ./network_datasets/api_dataset/south/test
[21-03-24 19:57:44 :INFO    ] - Post-processing 2020-04-01
[21-03-24 19:57:44 :INFO    ] - Post-processing 2020-04-02
[21-03-24 19:57:44 :INFO    ] - Dataset arr shape: (2, 432, 432, 7, 2)
[21-03-24 19:57:44 :INFO    ] - Applying active grid cell masks
[21-03-24 19:57:44 :INFO    ] - Land masking the forecast output
[21-03-24 19:57:44 :INFO    ] - Applying zeros to land mask
[21-03-24 19:57:44 :INFO    ] - Saving to ./results/predict/custom_run_forecast.nc


___
## 6. Visualisation

Now that we have a prediction, we can visualise the binary sea ice concentration using some of the built-in tools in IceNet that utilise `cartopy` and `matplotlib`.

(Note: There are also some scripts in the [icenet-pipeline](https://github.com/icenet-ai/icenet-pipeline) repository that enable plotting common results such as `produce_op_assets.sh`)

Here, we are loading the prediction netCDF file we've just created in the previous step.

We are also using the `Masks` class from IceNet to create a land mask region that will mask out the land regions in the forecast plot.

In [31]:
from icenet.plotting.video import xarray_to_video as xvid
from icenet.data.sic.mask import Masks
from IPython.display import HTML, display
import xarray as xr, pandas as pd, datetime as dt

def plot_result(file_path):
    # Load our output prediction file
    ds = xr.open_dataset(file_path)

    # Get land mask to mask these regions in final plot
    land_mask = Masks(south=True, north=False).get_land_mask()

    # We obtain the start date of the forecast we would like to plot
    forecast_date = ds.time.values[0]
    print(forecast_date)

    fc = ds.sic_mean.isel(time=0).drop_vars("time").rename(dict(leadtime="time"))
    fc['time'] = [pd.to_datetime(forecast_date) \
                + dt.timedelta(days=int(e)) for e in fc.time.values]

    anim = xvid(fc, 15, figsize=4, mask=land_mask)
    display(HTML(anim.to_jshtml()))

Now, we can use the built in plotting tool to visualise our forecast.

In [32]:
plot_result("results/predict/custom_run_forecast.nc")

INFO:root:Inspecting data
INFO:root:Initialising plot
INFO:root:Animating
INFO:root:Not saving plot, will return animation
INFO:matplotlib.animation:Animation.save using <class 'matplotlib.animation.HTMLWriter'>


2020-04-01T00:00:00.000000000


___
## Summary

This notebook has attempted to illustrate the workflow implementations of the CLI as well as highlight the flexibility of direct integration using it. Ultimately, library usage is the only way to achieve truly novel and flexible usage, the CLI is for convenience of running existing pipelines without having to manually implement complex scripts. 

The key to leveraging the benefits of both of these interfaces being provided is to consider using the following workflow:

* Get your environment(s) set up, be they research, development or production
* Use the existing CLI implementations to seed the data stores and get baseline networks operational
* Start to customise the existing operations via custom calls to the API, for example by downloading new variables or adding extra analysis to training/prediction runs
* If researching, consider [extending the functionality of the API to include revised or completely new implementations, such as additional data sources](05.library_extension.ipynb)

This last point brings us to the topic of the last of the introductory notebooks. 

## Version
- IceNet Codebase: v0.2.8